In [1]:
import csv
import json
import re
import pandas as pd

In [2]:
def convert_leasing_csv_to_json(csvFilePath, jsonFilePath):
    '''Converts leasing csv file into json'''
    df = pd.read_csv(csvFilePath)
    data = {}

    is_lost_reasons = False
    portfolio_key = None
    building_key = None
    LOST_REASONS = "Lost reasons"

    for index, row in df.iterrows():
        first_column_value = row[0]
        if not pd.isnull(first_column_value):
            match_portfolio = re.search(rf'Portfolio - ', first_column_value)
            match_building = re.search(rf'Building', first_column_value)
            match_lost_reasons = re.search(rf'{LOST_REASONS}', first_column_value)
            if match_lost_reasons:
                is_lost_reasons = True
                data[portfolio_key][building_key].setdefault(LOST_REASONS, {})
                continue


            if match_portfolio:
                portfolio_key = f"Portfolio-{first_column_value[match_portfolio.end():]}"
                data.setdefault(portfolio_key, {})
            elif match_building:
                # If matched a new building, reset is_lost_reasons
                building_key = first_column_value
                data[portfolio_key].setdefault(building_key, {})
                # Reset initial values
                is_lost_reasons = False
                continue
            elif portfolio_key and building_key:
                if is_lost_reasons:
                    # If is_lost_reasons, add new object within the nested "lost_reasons" object
                    key = first_column_value[3:]
                    data[portfolio_key][building_key][LOST_REASONS][key] = row.tolist()[1:]
                else:
                    # Otherwise, add new object within the nested building_key object
                    data[portfolio_key][building_key][first_column_value] = row.tolist()[1:]

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

In [3]:
convert_leasing_csv_to_json(r'data/csvs/insights sprint  - Leasing.csv', 'output/leasingData.json')

In [4]:
def convert_management_or_tenant_csv_to_json(csvFilePath, jsonFilePath):
    '''Converts management or tenant csv files into json'''
    
    df = pd.read_csv(csvFilePath)
    data = {}

    third_level_key = None
    portfolio_key = None
    building_key = None


    for index, row in df.iterrows():
        first_column_value = row[0]
        if not pd.isnull(first_column_value):
            match_portfolio = re.search(rf'Portfolio - ', first_column_value)
            match_building = re.search(rf'Building', first_column_value)

            if match_portfolio:
                portfolio_key = f"Portfolio-{first_column_value[match_portfolio.end():]}"
                data.setdefault(portfolio_key, {})
            elif match_building:
                building_key = first_column_value
                data[portfolio_key].setdefault(building_key, {})
                # Reset initial values
                third_level_key = None
                continue
            elif portfolio_key and building_key:
                # If the 2nd column value is null, this indicates a new third level nested object
                if pd.isnull(row[1]):
                    # Hence, we set the third_level_key to the 1st column value
                    third_level_key = first_column_value
                    data[portfolio_key][building_key].setdefault(third_level_key, {})
                    # Then we continue to the next row
                    continue
                if third_level_key:
                    data[portfolio_key][building_key][third_level_key][first_column_value] = row.tolist()[1:]
                else:
                    data[portfolio_key][building_key][first_column_value] = row.tolist()[1:]

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

In [5]:
convert_management_or_tenant_csv_to_json(rf'data/csvs/insights sprint  - Management.csv', 'output/managementData.json')

In [6]:
convert_management_or_tenant_csv_to_json(rf'data/csvs/insights sprint  - Tenant.csv', 'output/tenantData.json')



In [7]:
def convert_marketing_csv_to_json(csvFilePath, jsonFilePath):
    '''Converts marketing csv file into json'''
    df = pd.read_csv(csvFilePath)
    data = {}
    
    current_channel_key = None
    portfolio_key = None
    building_key = None
    CHANNELS = "Channels"
    ALL_CHANNELS = {'Zumper', 'Padmapper', 'Zillow', 'Facebook', 'Appartments.com', 'Direct (website)'}


    for index, row in df.iterrows():
        first_column_value = row[0]
        
            
        
        if not pd.isnull(first_column_value):
#             print(first_column_value.split().join(" "))
            # Skip row if first column value is Channels
            if re.search(rf'Channels', first_column_value):
                continue
                
            match_portfolio = re.search(rf'Portfolio - ', first_column_value)
            match_building = re.search(rf'Building', first_column_value)

            if match_portfolio:
                portfolio_key = f"Portfolio-{first_column_value[match_portfolio.end():]}"
                data.setdefault(portfolio_key, {})
            elif match_building:
                building_key = first_column_value
                data[portfolio_key].setdefault(building_key, {CHANNELS:{}})
                # Reset initial values
                current_channel_key = None
                continue
            elif portfolio_key and building_key:
                
                # If first_column_value matches one of the existing hardcoded channels, create a new nested object
                if first_column_value in ALL_CHANNELS:
                    current_channel_key = first_column_value
                    data[portfolio_key][building_key][CHANNELS].setdefault(current_channel_key, {})
                    continue
                    
                # Format row data such that blank cells are converted into ''
                row_data = [el if not pd.isnull(el) else '' for el in row.tolist()[1:]]
                if current_channel_key:
                    data[portfolio_key][building_key][CHANNELS][current_channel_key][first_column_value] = row_data
                else:
                    data[portfolio_key][building_key][first_column_value] = row_data

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

In [8]:
convert_marketing_csv_to_json(rf'data/csvs/insights sprint  - Data input (marketing) (1).csv', 'output/marketingData.json')
